In [1]:
import torch
x = torch.rand(5, 3)
print(x)

print(torch.cuda.is_available())

tensor([[0.6709, 0.7642, 0.2177],
        [0.5598, 0.2012, 0.8318],
        [0.0416, 0.1889, 0.4911],
        [0.2457, 0.7411, 0.6778],
        [0.4753, 0.9681, 0.7996]])
False
